In [103]:
import pandas as pd
import numpy as np

In [266]:
df=pd.read_csv('Amazon-products.csv')

In [267]:
df = df.drop('Unnamed: 0', axis=1)


In [268]:
df.isna().sum()

name                   0
main_category          0
sub_category           0
image                  0
link                   0
ratings           175794
no_of_ratings     175794
discount_price     61163
actual_price       17813
dtype: int64

In [269]:
df['no_of_ratings']=df['no_of_ratings'].fillna(0)
df['ratings']=df['ratings'].fillna(0)
df['discount_price']=df['discount_price'].fillna(0)
df=df.dropna()

In [270]:
print(df.duplicated().sum())
print(df.isna().sum())

0
name              0
main_category     0
sub_category      0
image             0
link              0
ratings           0
no_of_ratings     0
discount_price    0
actual_price      0
dtype: int64


In [271]:
# Removing the ₹ sign
df["discount_price"] = df["discount_price"].str.split(" ", expand = True).get(0).str.split("₹", expand = True).get(1)
df["actual_price"] = df["actual_price"].str.split(" ", expand = True).get(0).str.split("₹", expand = True).get(1)
# Change commas to dots and change the type to float
df['discount_price'] = df["discount_price"].str.replace(',', '').astype(float)
df["actual_price"] = df["actual_price"].str.replace(',', '').astype(float)

In [272]:
df['ratings'] = pd.to_numeric(df['ratings'], errors='coerce')
df = df.dropna(subset=['ratings'])

# Extract the digits and change the type to float
#df['ratings'] = df['ratings'].replace(['Get','FREE','₹68.99', '₹65','₹70', '₹100', '₹99', '₹2.99'], '0.0')
df['ratings'] = df["ratings"].astype(float)
df['ratings'].unique()

array([4.2, 4. , 4.1, 4.3, 3.9, 3.8, 3.5, 0. , 4.6, 3.3, 3.4, 3.7, 2.9,
       5. , 4.4, 3.6, 2.7, 4.5, 3. , 3.1, 3.2, 4.8, 4.7, 2.5, 1. , 2.6,
       2.8, 1.7, 1.8, 2.4, 4.9, 2.2, 1.6, 1.9, 2.3, 2. , 1.4, 2.1, 1.2,
       1.3, 1.5, 1.1])

In [273]:
df['no_of_ratings']=df['no_of_ratings'].str.replace(',','')
df['no_of_ratings'] = pd.to_numeric(df['no_of_ratings'], errors='coerce')

# drop any rows with NaN values in the 'no_of_ratings' column
df = df.dropna(subset=['no_of_ratings'])

# convert the 'no_of_ratings' column to integer values
df['no_of_ratings'] = df['no_of_ratings'].astype(int)

In [274]:
len(df)

362797

In [275]:
result = {}

# loop over each unique main category value
for main_cat in df['main_category'].unique():
    
    # get the unique sub category values for this main category
    sub_cats = df.loc[df['main_category'] == main_cat, 'sub_category'].unique().tolist()
    
    # add the main category and its sub categories to the result dictionary
    result[main_cat] = sub_cats

In [ ]:
import json

# create a dictionary

# open a file for writing
with open('categories.json', 'w') as f:
    # write the dictionary to the file in JSON format
    json.dump(result, f)

In [276]:
len(df)

362797

In [277]:
vote_counts = df[df['no_of_ratings'].notnull()]['no_of_ratings'].astype('int')
vote_averages = df[df['ratings'].notnull()]['ratings'].astype('int')
C = vote_averages.mean()

In [278]:
m = df.no_of_ratings.quantile(0.85)


In [279]:
qualified = df[(df['no_of_ratings'] >= m) & (df['no_of_ratings'].notnull()) & (df['ratings'].notnull())][['name', 'main_category', 'sub_category', 'image', 'link', 'ratings','no_of_ratings','discount_price','actual_price']]
qualified['no_of_ratings'] = qualified['no_of_ratings'].astype('int')
qualified['ratings'] = qualified['ratings'].astype('float')
qualified.shape

(54420, 9)

In [280]:
def weighted_rating(x):
    v = x['no_of_ratings']
    R = x['ratings']
    return (v/(v+m) * R) + (m/(m+v) * C)

In [281]:
qualified['wr'] = qualified.apply(weighted_rating, axis=1)

In [282]:
qualified = qualified.sort_values('wr', ascending=False)

In [283]:
qualified['discount_price']=qualified['discount_price'].fillna(value=0)

In [284]:
qualified.to_csv('final_data.csv', index=False)


In [295]:
def build_chart(main_category,sub_category=None,percentile=0.85):
    df1 = df[df['main_category'] == main_category]
    if sub_category is not None:
        df1 = df1[df1['sub_category'] == sub_category]
        category = sub_category
    else:
        category = main_category
    vote_counts = df1[df1['no_of_ratings'].notnull()]['no_of_ratings'].astype('int')
    vote_averages = df1[df1['ratings'].notnull()]['ratings'].astype('int')
    C = vote_averages.mean()
    m = vote_counts.quantile(percentile)
    
    qualified = df1[(df1['no_of_ratings'] >= m) & (df1['no_of_ratings'].notnull()) & (df1['ratings'].notnull())][['name', 'main_category', 'sub_category', 'image', 'link', 'ratings','no_of_ratings','discount_price','actual_price']]
    qualified['no_of_ratings'] = qualified['no_of_ratings'].astype('int')
    qualified['ratings'] = qualified['ratings'].astype('float')
    
    qualified['wr'] = qualified.apply(lambda x: (x['no_of_ratings']/(x['no_of_ratings']+m) * x['ratings']) + (m/(m+x['no_of_ratings']) * C), axis=1)
    qualified = qualified.sort_values('wr', ascending=False).head(10)
    
    return qualified

In [297]:
build_chart('appliances','All Appliances')


,name,main_category,sub_category,image,link,ratings,no_of_ratings,discount_price,actual_price,wr
1422,"Cascade Mini Portable Air Cooler,Personal Spac...",appliances,All Appliances,https://m.media-amazon.com/images/I/61Vnoy0m+Q...,https://www.amazon.in/Cascade-Portable-Persona...,4.8,52281,2999.0,4999.0,4.775330
7427,iRobot Authentic Replacement Parts- Clean Base...,appliances,All Appliances,https://m.media-amazon.com/images/I/71VcMpQAVs...,https://www.amazon.in/Authentic-Replacement-Au...,4.8,18941,1999.0,6999.0,4.734168
3975,"LEVOIT Mini Portable Air Cooler,Personal Space...",appliances,All Appliances,https://m.media-amazon.com/images/I/71lOGeZZQn...,https://www.amazon.in/LEVOIT-Conditioner-moldp...,4.8,13283,2499.0,3999.0,4.708163
9697,"Dash DMW001RD Mini Waffle Maker, Red",appliances,All Appliances,https://m.media-amazon.com/images/W/IMAGERENDE...,https://www.amazon.in/Red-Dash-DMW001RD-Waffle...,4.7,53915,3278.0,4689.0,4.677957
3141,AmazonBasics 2 slice Toaster with Stainless St...,appliances,All Appliances,https://m.media-amazon.com/images/I/713n1fmP3U...,https://www.amazon.in/Amazon-Basics-Toaster-se...,4.7,19190,2199.0,2999.0,4.640121
4851,"Instant Pot Air Fryer, Vortex 4 Litre, Touch C...",appliances,All Appliances,https://m.media-amazon.com/images/W/IMAGERENDE...,https://www.amazon.in/Instant-Vortex-One-Touch...,4.7,14904,22200.0,25541.0,4.624025
6592,Irobot Replacement Parts- Braava Jet Hard Floo...,appliances,All Appliances,https://m.media-amazon.com/images/I/61lS3YCy3x...,https://www.amazon.in/Authentic-Replacement-Cl...,4.7,10573,NaN,1490.0,4.595692
6677,Irobot Replacement Parts- Braava Jet Hard Floo...,appliances,All Appliances,https://m.media-amazon.com/images/W/IMAGERENDE...,https://www.amazon.in/Authentic-Replacement-Cl...,4.7,10573,NaN,1490.0,4.595692
5427,Karcher Filter Bag Fleece PVC Set Packaged 4X ...,appliances,All Appliances,https://m.media-amazon.com/images/I/81yk65EfHs...,https://www.amazon.in/Karcher-Packaged-Cleaner...,4.7,9635,1199.0,1476.0,4.586527
1738,"Instant Pot Air Fryer, Vortex 4 Litre Essentia...",appliances,All Appliances,https://m.media-amazon.com/images/W/IMAGERENDE...,https://www.amazon.in/Instant-Vortex-4QT-EvenC...,4.8,4302,6993.0,20088.0,4.553780


In [ ]:
df.to_csv('final_data.csv', index=False)
